In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [3]:
raw = get_machines()

In [5]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_34460\1583430346.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_34460\1583430346.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_34460\1583430346.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [6]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'location']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
6767,RTX A6000,[0/2],5.01,0.4,5.4,10,126,1215,1892.0,1,...,41.300000,12.0,525.125.06,3,16,NaN,11.5,Core™ i9-10900X,WS X299 SAGE,"{'country': 'CA', 'location': 'Vancouver, Brit..."
11937,H100 PCIE,[1/2],3.00,0.4,34.3,64,118,1561,4428.0,1,...,885.680078,12.2,535.54.03,4,16,NaN,19.8,EPYC 7742 64C,MZ52,"{'country': 'US', 'location': 'Ogden, Utah', '..."
11938,RTX 3090,[0/1],2.17,0.4,16.6,6,31,808,1645.0,1,...,526.100000,12.0,525.125.06,3,16,NaN,12.4,Core™ i5-9600K,B365 HD3,"{'country': 'US', 'location': 'Cleveland, Ohio..."
12027,RTX 3080 Ti,[0/1],1.92,0.4,3.1,2,16,65,156.0,1,...,NaN,12.0,525.125.06,2,1,NaN,0.3,Pentium® G3220,,"{'country': 'TH', 'location': 'Chiang Mai', 'l..."
12011,A100 PCIE,[0/1],1.87,0.3,32.1,24,216,804,2678.0,1,...,2593.552753,12.2,535.54.03,4,16,NaN,25.4,EPYC 7V13 64C,Virtual Machine,"{'country': 'GB', 'location': 'London, England..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10755,RTX 4090,[4/4],0.50,0.4,89.3,8,31,115,1481.0,1,...,834.500000,12.0,525.105.17,3,8,NaN,6.3,Ryzen TR 2950X 16C,X399 AORUS XTREME,"{'country': 'CA', 'location': 'Montreal, Quebe..."
9051,A40,[1/10],0.50,0.4,39.7,1,13,69,1250.0,1,...,477.400000,12.0,525.125.06,3,16,1.0,11.9,EPYC 7252 8C,H11DSi,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
9053,A40,[5/5],0.50,NaN,60.4,3,50,122,1305.0,1,...,558.200000,11.7,515.86.01,3,16,1.0,6.1,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."


In [7]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
11930,RTX 3080,[0/1],0.100,0.40,217.6,4,16,75,2402.0,1,...,60.100000,12.2,535.86.05,3,16,NaN,12.4,Core™ i5-7400,H110M,"{'country': 'RU', 'location': 'Bratsk, Irkutsk..."
11900,RTX 3080 Ti,[0/3],0.146,0.05,169.7,10,21,245,3350.0,1,...,374.500000,12.0,525.125.06,4,4,NaN,6.0,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11637,RTX 3080 Ti,[3/3],0.146,0.05,169.0,10,21,457,3117.0,1,...,451.800000,12.0,525.125.06,4,4,NaN,6.0,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11613,RTX 3080 Ti,[1/2],0.150,0.05,166.5,18,63,608,3256.0,1,...,393.300000,12.0,525.125.06,3,16,NaN,11.8,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11536,RTX 3090,[4/4],0.230,0.10,129.1,14,63,297,1160.0,2,...,810.400000,12.0,525.105.17,3,16,NaN,10.7,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11480,RTX 3090,[2/2],0.200,0.00,145.2,16,63,174,3531.0,1,...,354.700000,12.0,525.125.06,3,8,NaN,6.4,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11395,RTX 3090,[4/4],0.230,0.10,129.3,14,63,251,1829.0,2,...,877.867057,12.0,525.125.06,3,16,NaN,10.6,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11385,RTX 4090,[6/6],0.395,0.40,147.3,10,42,191,3400.0,1,...,515.900000,12.0,525.105.17,4,16,NaN,23.3,EPYC 7532 32C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
11230,RTX 4090,[2/2],0.400,0.20,189.9,64,63,818,1580.0,1,...,501.900000,12.0,525.105.17,4,16,NaN,23.1,EPYC 7B13 64C,ROMED8,"{'country': 'RU', 'location': 'Fryazino, Mosco..."
